In [ ]:
pip install pandas numpy scikit-learn pillow ipywidgets gradio

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import gradio as gr
import os
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:

def load_data():
    try:
        df = pd.read_csv("archive/main.csv")
        df['is_male'] = df['gender'].apply(lambda x: 1 if x.lower() == 'male' else 0)
        df['is_female'] = df['gender'].apply(lambda x: 1 if x.lower() == 'female' else 0)
        if 'feature' in df.columns:
            all_features = set()
            for features in df['feature'].str.split(','):
                if isinstance(features, list):
                    all_features.update([f.strip().lower() for f in features])
            
            for feature in all_features:
                if feature:
                    col_name = f"is_{feature.replace(' ', '_')}"
                    df[col_name] = df['feature'].str.contains(feature, case=False).astype(int)
        
        return df
    except Exception as e:
        print(f"Error: {e}")
        data = {
            "file_name": ["amitabh.jpg", "priyanka.jpg", "modi.jpg"],
            "person_name": ["Amitabh Bachchan", "Priyanka Chopra", "Narendra Modi"],
            "gender": ["male", "female", "male"],
            "total_of_each": [100, 85, 120],
            "feature": ["actor,bollywood", "actor,singer", "politician,pm"]
        }
        return pd.DataFrame(data)

df = load_data()
features = [col for col in df.columns if col.startswith('is_')]
X = df[features]
y = df['person_name']
model = DecisionTreeClassifier(max_depth=5)
model.fit(X, y)

In [ ]:
def get_image_path(person_name):
    try:
        gender = df[df['person_name'] == person_name]['gender'].values[0].lower()
        filename = df[df['person_name'] == person_name]['file_name'].values[0]
        
        image_dir = Path("archive/images") / gender
        for ext in ['.jpg', '.jpeg', '.png']:
            image_path = image_dir / (filename.split('.')[0] + ext)
            if image_path.exists():
                return str(image_path)
        image_path = Path("archive/images") / filename
        if image_path.exists():
            return str(image_path)
            
    except Exception as e:
        print(f"Image not found for {person_name}: {e}")
    return None
def guess_person(*answers):
    input_data = [1 if ans else 0 for ans in answers]
    prediction = model.predict([input_data])[0]
    proba = model.predict_proba([input_data])[0]
    confidence = max(proba) * 100
    image_path = get_image_path(prediction)
    if image_path:
        image = Image.open(image_path)
    else:
        image = None
    top_3_idx = np.argsort(proba)[-3:][::-1]
    top_3 = [(model.classes_[i], f"{proba[i]*100:.1f}%") for i in top_3_idx]
    
    return image, prediction, f"{confidence:.1f}%", top_3
question_components = []
for i, feature in enumerate(features):
    question = feature.replace('is_', '').replace('_', ' ').title() + "?"
    question_components.append(
        gr.Checkbox(label=question, info="Check if Yes, leave unchecked for No")
    )
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🧞 Indian Celebrity Guesser")
    gr.Markdown("Think of a famous Indian personality and answer these questions:")
    
    with gr.Row():
        with gr.Column():
            for component in question_components:
                component.render()
            
            submit_btn = gr.Button("Guess Who!", variant="primary")
        
        with gr.Column():
            image_output = gr.Image(label="Predicted Celebrity")
            name_output = gr.Textbox(label="Name")
            confidence_output = gr.Textbox(label="Confidence")
            
            with gr.Accordion("See other possibilities", open=False):
                top3_output = gr.Dataframe(
                    headers=["Name", "Confidence"],
                    datatype=["str", "str"]
                )
    
    submit_btn.click(
        guess_person,
        inputs=question_components,
        outputs=[image_output, name_output, confidence_output, top3_output]
    )
    
    gr.Examples(
        examples=[
            [True, False, False, True, False],  
            [False, True, True, False, False]   
        ],
        inputs=question_components,
        outputs=[image_output, name_output, confidence_output, top3_output],
        fn=guess_person,
        cache_examples=True
    )

demo.launch()